In [1]:
from pyarrow import fs, hdfs
import pyarrow.csv as pv
import pyarrow.compute as pc
import collections
import re
hdfs.HadoopFileSystem("hdfs://namenode:9000")
conexao = fs.HadoopFileSystem(host="namenode", port=9000)

/tmp/ipykernel_143/1205232693.py:6: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  hdfs.HadoopFileSystem("hdfs://namenode:9000")


In [2]:
conexao.get_file_info(fs.FileSelector('/', recursive=True))

[]

In [3]:
conexao.create_dir('/pasta_avo1')

In [4]:
caminho_arquivo_origem = '/home/jovyan/work/sentiment140.csv'
caminho_arquivo_destino = '/pasta_avo1/sentiment140.csv'

with conexao.open_output_stream(caminho_arquivo_destino) as stream:
    stream.write(open(caminho_arquivo_origem, 'rb').read())

In [5]:
with conexao.open_input_stream(caminho_arquivo_destino) as f:
    print(f.read(500))

b'\xef\xbb\xbftarget;ids;date;flag;user;text\r\n0;1467810369;Mon Apr 06 22:19:45 PDT 2009;NO_QUERY;_TheSpecialOne_;"@switchfoot http://twitpic.com/2y1zl - Awww, that\'s a bummer.  You shoulda got David Carr of Third Day to do it. ;D"\r\n0;1467810672;Mon Apr 06 22:19:49 PDT 2009;NO_QUERY;scotthamilton;is upset that he can\'t update his Facebook by texting it... and might cry as a result  School today also. Blah!\r\n0;1467810917;Mon Apr 06 22:19:53 PDT 2009;NO_QUERY;mattycus;@Kenichan I dived many times for the ball'


In [6]:
table = pv.read_csv(conexao.open_input_file(caminho_arquivo_destino), parse_options=pv.ParseOptions(delimiter=';'))
print('table:')
print(table)

table:
pyarrow.Table
target: int64
ids: int64
date: string
flag: string
user: string
text: string
----
target: [[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0],...,[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0]]
ids: [[1467810369,1467810672,1467810917,1467811184,1467811193,...,1469870096,1469870102,1469870284,1469870477,1469870674],[1469870883,1469870966,1469871462,1469871517,1469871581,...,1554016398,1554016920,1554016950,1554016980,1554017010],...,[2001145720,2001145923,2001145946,2001146065,2001146082,...,2002774308,2002774545,2002774875,2002774938,2002774950],[2002775348,2002775611,2002775764,2002775814,2002775834,...,2003265997,2003266067,2003266125,2003266274,2003266290]]
date: [["Mon Apr 06 22:19:45 PDT 2009","Mon Apr 06 22:19:49 PDT 2009","Mon Apr 06 22:19:53 PDT 2009","Mon Apr 06 22:19:57 PDT 2009","Mon Apr 06 22:19:57 PDT 2009",...,"Tue Apr 07 07:39:44 PDT 2009","Tue Apr 07 07:39:43 PDT 2009","Tue Apr 07 07:39:45 PDT 2009","Tue Apr 07 07:39:48 PDT 2009","Tue Apr 07 0

In [7]:
import pyarrow.csv as pv

table = pv.read_csv(conexao.open_input_file(caminho_arquivo_destino), parse_options=pv.ParseOptions(delimiter=';'))

tweets = table.column('text').to_pylist()

word_counts = collections.Counter()

for tweet in tweets:
    words = re.findall(r'\b\w+\b', tweet.lower())
    word_counts.update(words)

print("palavras mais frequentes")
for word, count in word_counts.most_common(10):
    print(f'{word}: {count}')

palavras mais frequentes
i: 226118
to: 127466
the: 104841
my: 74810
a: 73919
it: 63174
and: 62151
is: 52001
t: 49829
in: 46649


In [8]:
mention_counts = collections.Counter()

for tweet in tweets:
    mentions = re.findall(r'@\w+', tweet)
    mention_counts.update(mentions)

print("Usuarios mais mencionados:")
for mention, count in mention_counts.most_common(10):
    print(f'{mention}: {count}')

Usuarios mais mencionados:
@mileycyrus: 719
@tommcfly: 664
@ddlovato: 417
@JonathanRKnight: 221
@taylorswift13: 200
@mitchelmusso: 192
@DavidArchie: 185
@jordanknight: 159
@selenagomez: 144
@DonnieWahlberg: 127


In [9]:
hashtag_counts = collections.Counter()

for tweet in tweets:
    hashtags = re.findall(r'#\w+', tweet)
    hashtag_counts.update(hashtags)

print("Hashtags mais frequentes:")
for hashtag, count in hashtag_counts.most_common(10):
    print(f'{hashtag}: {count}')

Hashtags mais frequentes:
#fb: 373
#fail: 115
#asot400: 113
#bgt: 73
#1: 70
#2: 64
#followfriday: 61
#NOME: 55
#myweakness: 54
#e3: 52


In [10]:
#desafio 1:
from datetime import datetime
dates = table.column('date').to_pylist()
days_of_week = [datetime.strptime(date, '%a %b %d %H:%M:%S PDT %Y').strftime('%A') for date in dates]
words_by_day = {day: collections.Counter() for day in set(days_of_week)}

for tweet, day in zip(tweets, days_of_week):
    words = re.findall(r'\b\w+\b', tweet.lower())
    words_by_day[day].update(words)

for day, counter in words_by_day.items():
    print(f"Palavras mais frequentes em {day}:")
    for word, count in counter.most_common(10):
        print(f'{word}: {count}')


Palavras mais frequentes em Saturday:
i: 49491
to: 26497
the: 22172
my: 16167
a: 16040
and: 13279
it: 13167
is: 11001
t: 10702
in: 9938
Palavras mais frequentes em Thursday:
i: 10428
to: 5672
the: 4528
my: 3471
a: 3300
it: 2916
and: 2693
t: 2293
is: 2258
in: 2049
Palavras mais frequentes em Monday:
i: 52011
to: 30259
the: 24993
my: 17401
a: 17171
it: 15179
and: 14352
is: 12323
t: 11538
in: 11036
Palavras mais frequentes em Tuesday:
i: 16566
to: 9768
the: 8274
a: 5723
my: 5574
it: 4951
and: 4823
is: 4367
t: 3834
in: 3803
Palavras mais frequentes em Wednesday:
i: 1537
to: 893
the: 680
my: 511
a: 483
and: 386
is: 367
t: 366
it: 359
in: 327
Palavras mais frequentes em Friday:
i: 34997
to: 18852
the: 15488
my: 11409
a: 11306
it: 9621
and: 9218
t: 7577
is: 7543
in: 7077
Palavras mais frequentes em Sunday:
i: 61088
to: 35525
the: 28706
my: 20277
a: 19896
and: 17400
it: 16981
is: 14142
t: 13519
in: 12419


In [12]:
#desafio 2:
hours_of_day = [datetime.strptime(date, '%a %b %d %H:%M:%S PDT %Y').strftime('%H') for date in dates]
hashtags_by_hour = {hour: collections.Counter() for hour in set(hours_of_day)}

for tweet, hour in zip(tweets, hours_of_day):
    hashtags = re.findall(r'#\w+', tweet)
    hashtags_by_hour[hour].update(hashtags)

for hour, counter in hashtags_by_hour.items():
    print(f"Hashtags mais frequentes na hora {hour}:00:")
    for hashtag, count in counter.most_common(10):
        print(f'{hashtag}: {count}')

Hashtags mais frequentes na hora 18:00:
#mmwanted: 12
#myweakness: 7
#fb: 7
#e3: 6
#sigjeans: 5
#BEAtwittyparty: 5
#NOME: 4
#spymaster: 3
#mtvmovieawards: 3
#ChuckMeMondays: 3
Hashtags mais frequentes na hora 02:00:
#fb: 29
#fail: 12
#bradiewebb: 11
#asot400: 9
#myweakness: 9
#marsiscoming: 8
#f1: 6
#mcflyinmanila: 5
#bck5: 4
#NOME: 4
Hashtags mais frequentes na hora 12:00:
#bgt: 9
#fb: 8
#BGT: 6
#asot400: 5
#twpp: 5
#e3: 4
#xboxe3: 4
#thingsmummysaid: 3
#XboxE3: 3
#Penguins: 2
Hashtags mais frequentes na hora 01:00:
#fb: 26
#bradiewebb: 16
#marsiscoming: 15
#fail: 11
#bck5: 6
#f1: 5
#asot400: 5
#NOME: 4
#followfriday: 4
#2: 4
Hashtags mais frequentes na hora 04:00:
#fb: 23
#ShaunJumpnow: 8
#fail: 6
#Tweetie: 4
#tweetie: 4
#1: 4
#eurovision: 4
#myweakness: 4
#asot400: 3
#2: 3
Hashtags mais frequentes na hora 15:00:
#asot400: 14
#fb: 9
#fail: 4
#chesterday: 4
#e3: 4
#MCRchat: 3
#kitchenfire: 2
#followfriday: 2
#6: 2
#itsucks: 2
Hashtags mais frequentes na hora 07:00:
#fb: 23
#andyhurley